# Opportunity 2

## Imports

In [30]:
import pandas as pd
import numpy as np
from pathlib import Path

## Data setup

In [31]:
data_dir = Path('data')
stores_data = {}

for parquet_file in data_dir.glob('*.parquet*'):
    file_name = str(parquet_file).replace('data\sales-', '').replace('.parquet.gzip', '')
    stores_data[file_name] = {'sales_df': pd.read_parquet(parquet_file).drop(['SALES_DIST', 'SOLD_TO', 'SHIP_TO'], axis=1)}
    
stores_data

{'data/sales-7017139': {'sales_df':            BILL_DATE  MATERIAL  PIEZAS
  327623    2019-03-04       425       1
  342880    2019-03-04     12080       1
  343841    2019-03-04      2639       1
  347866    2019-03-04     12026       1
  467020    2019-03-04     29772       2
  ...              ...       ...     ...
  186696013 2021-03-29     12793       2
  186696014 2021-03-29     22054       4
  186908262 2021-03-29     43004       1
  186915466 2021-03-29     16640       1
  186925567 2021-03-29     36717       2
  
  [4379 rows x 3 columns]},
 'data/sales-7003984': {'sales_df':            BILL_DATE  MATERIAL  PIEZAS
  8511      2019-03-03      2641       1
  8512      2019-03-03     12916       1
  8513      2019-03-03      2464       1
  9478      2019-03-03     11074       1
  9479      2019-03-03     20306       1
  ...              ...       ...     ...
  141730332 2020-09-17     31907       2
  141730333 2020-09-17     36526       2
  141730334 2020-09-17     36879       1

## Get first and last date for each store

In [32]:
for store, data in stores_data.items():
    df_sorted_by_dates = data['sales_df'].sort_values(by='BILL_DATE')
    data['oldest_date'] = df_sorted_by_dates.iloc[0]["BILL_DATE"]
    data['newest_date'] = df_sorted_by_dates.iloc[-1]["BILL_DATE"]

## Top 10 most sold products per store

In [33]:
for store, data in stores_data.items():
    product_sales = data['sales_df'].groupby(['MATERIAL'])['PIEZAS'].sum().sort_values(ascending=False)
    data['top_products'] = product_sales.head(10)

stores_data

{'data/sales-7017139': {'sales_df':            BILL_DATE  MATERIAL  PIEZAS
  327623    2019-03-04       425       1
  342880    2019-03-04     12080       1
  343841    2019-03-04      2639       1
  347866    2019-03-04     12026       1
  467020    2019-03-04     29772       2
  ...              ...       ...     ...
  186696013 2021-03-29     12793       2
  186696014 2021-03-29     22054       4
  186908262 2021-03-29     43004       1
  186915466 2021-03-29     16640       1
  186925567 2021-03-29     36717       2
  
  [4379 rows x 3 columns],
  'oldest_date': Timestamp('2019-03-04 00:00:00'),
  'newest_date': Timestamp('2021-03-29 00:00:00'),
  'top_products': MATERIAL
  4101     200
  13165    115
  4124     100
  19197     86
  4373      80
  4102      70
  9664      68
  3953      68
  3954      65
  4108      62
  Name: PIEZAS, dtype: int64},
 'data/sales-7003984': {'sales_df':            BILL_DATE  MATERIAL  PIEZAS
  8511      2019-03-03      2641       1
  8512      2019-0

## Get daily sales of top 10 products of each store

In [34]:
for store, data in stores_data.items():
    idx = pd.date_range(data['oldest_date'], data['newest_date'])
    data['top_products_sales'] = {}
    for product, sale in data['top_products'].iteritems():
        product_series = data['sales_df'][data['sales_df']["MATERIAL"] == product]
        product_series = product_series.groupby('BILL_DATE').PIEZAS.sum().to_frame(name = 'PIEZAS')

        product_series.index = pd.DatetimeIndex(product_series.index)
        product_series = product_series.reindex(idx, fill_value=0)
        product_series.index.name = "BILL_DATE"
        product_series = product_series.reset_index()
        data['top_products_sales'][product] = product_series

stores_data

{'data/sales-7017139': {'sales_df':            BILL_DATE  MATERIAL  PIEZAS
  327623    2019-03-04       425       1
  342880    2019-03-04     12080       1
  343841    2019-03-04      2639       1
  347866    2019-03-04     12026       1
  467020    2019-03-04     29772       2
  ...              ...       ...     ...
  186696013 2021-03-29     12793       2
  186696014 2021-03-29     22054       4
  186908262 2021-03-29     43004       1
  186915466 2021-03-29     16640       1
  186925567 2021-03-29     36717       2
  
  [4379 rows x 3 columns],
  'oldest_date': Timestamp('2019-03-04 00:00:00'),
  'newest_date': Timestamp('2021-03-29 00:00:00'),
  'top_products': MATERIAL
  4101     200
  13165    115
  4124     100
  19197     86
  4373      80
  4102      70
  9664      68
  3953      68
  3954      65
  4108      62
  Name: PIEZAS, dtype: int64,
  'top_products_sales': {4101:      BILL_DATE  PIEZAS
   0   2019-03-04       0
   1   2019-03-05       0
   2   2019-03-06       0
   